In [8]:
from pynytimes import NYTAPI
import datetime
from newspaper import Article
import requests
import pandas as pd
import time
api_key="2z5QXdWZFM4Qe9scDJm12YzOiJvaj357"
api_key1='Vmyj1hoceXUZMw69FVT4qpXKdyS1NoBd'
nyt = NYTAPI(api_key)
def get_all_comments(api_key, nyt_article):
        '''
        Input:
        api_key = Your API KEY (string)
        nyt_article = link to the nyt article that has comments (string)
        Output:
        Pandas DataFame
        This function will extract all the comments out of a NYT article, and stores it in a DataFrame
        '''
        import time
        #set offset to 0
        offset = 0

        #At some point the json_data['results']['comments'] will return an empty list
        comment_body_check = ['test']

        #empty dataframe to store data
        comment_df = pd.DataFrame()

        while comment_body_check != []:

             #URL for scraping data
            request_url = f'https://api.nytimes.com/svc/community/v3/user-content/url.json?api-key={api_key}&offset={offset}&url={nyt_article}'
            #Get data
            r = requests.get(request_url)

            #Conver to JSON
            json_data = r.json()

            #Single out the comment section, it becomes a list
            comment_json = json_data['results']['comments']

            comment_body_check = comment_json

            #create list
            comment_list = []
            df = pd.DataFrame(comment_json, columns = ['commentID',
                                                       'userDisplayName',
                                                       'userLocation',
                                                       'commentBody',
                                                       'recommendations',
                                                       'replyCount'])                                            

            #append 25 scraped commentt
            comment_df = comment_df.append(df)
            #6 second pause, so that the API doesn't get overwhelmed
            time.sleep(4)

        #resetting index for a clean output
        comment_df.reset_index(inplace = True)

        return comment_df


In [9]:
keywords=['facial recognition Bias','face recognition Bias','Deep learning Bias','Artificial Intelligence Bias','AI Bias','Machine Learning Bias','ML bias'
           'Algorithmic bias', 'Algorithm bias','facial recognition discrimination', 'face recognition discrimination','Deep learning discrimination',
          'Artificial Intelligence discrimination','AI discrimination','Machine Learning discrimination','ML discrimination',
          'Algorithmic discrimination', 'Algorithm discrimination'
          ]

In [37]:
df_articles = pd.DataFrame()
for i in keywords:
    articles = nyt.article_search(
                query = i,
                results = 50,
                dates = {
                    "begin": datetime.datetime(2017,1,1),
                    "end": datetime.datetime(2017,12,31)
                })
    df_articles1 = pd.DataFrame.from_dict(articles, orient='columns')
    df_articles = df_articles.append(df_articles1)
dfweb = df_articles.web_url
dfweb = dfweb.drop_duplicates()
len(dfweb)

165

In [38]:
dfweb = dfweb.to_frame()
dfweb.reset_index(inplace=True,drop=True)
dfweb

,web_url
0,https://www.nytimes.com/2017/12/21/us/politics...
1,https://www.nytimes.com/2017/12/02/opinion/sun...
2,https://www.nytimes.com/2017/08/23/theater/ran...
3,https://www.nytimes.com/2017/09/09/opinion/sun...
4,https://www.nytimes.com/2017/08/29/t-magazine/...
...,...
160,https://www.nytimes.com/2017/12/19/nyregion/ne...
161,https://www.nytimes.com/2017/04/05/your-money/...
162,https://www.nytimes.com/2017/09/20/business/me...
163,https://www.nytimes.com/2017/01/07/opinion/sun...


In [39]:
dfweb.to_csv('2017.csv',index=False)

In [40]:
df_articles=pd.read_csv('total_url.csv')
urllist=df_articles.web_url.tolist()
len(urllist)

691

In [44]:
comment_df = pd.DataFrame()
offset=[0,25,50,75,100,125,150,175,200]
a=0

for i in urllist: 
    for os in offset:
        request_url=f'https://api.nytimes.com/svc/community/v3/user-content/url.json?api-key={api_key}&offset={offset}&url={i}'
        r = requests.get(request_url)
        json_data = r.json()

        try:
            comment_json = json_data['results']['comments']
            comment_body_check = comment_json
            df = pd.DataFrame(comment_json, columns = ['commentID',
                                                               'userDisplayName',
                                                               'userLocation',
                                                               'commentBody',
                                                               'recommendations',
                                                               'replyCount'
                                                               ]) 
            urlindex=urllist.index(i)
            if urlindex==0:
                df.to_csv('comments.csv',index=False)
            else:
                df.to_csv("comments.csv", mode='a', index=False, header=False)
        except:
            a+=1
            continue
        time.sleep(5)
print(a)

5375
